In [ ]:
import pandas as pd
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

df = pd.read_csv("train.csv")


In [ ]:
def get_title(name):
    if "." in name:
        return name.split(',')[1].split(".")[0].strip()
    else:
        return "no title"

In [ ]:
def shorter(x):
    title = x["Title"]
    if title in ['Capt','Col','Major']:
        return "Officer"
    elif title in ['Jonkeer', 'Don','the countess','Dona','Lady','Sir']:
        return "Royality"
    elif title == 'Master':
        return "Mrs"
    elif title in ['Mile', 'Ms']:
        return "Ms"
    else:
        return title

In [ ]:
df["Title"] = df['Name'].map(lambda x : get_title(x))

In [ ]:
df["Title"] = df.apply(shorter, axis=1)
df['Age'].fillna(df['Age'].median(), inplace=True)
df['Fare'].fillna(df['Fare'].median(), inplace=True)
df['Embarked'].fillna("S", inplace=True)
df.drop("Cabin", axis=1, inplace = True)
df.drop("Ticket", axis=1, inplace = True)
df.drop("Name", axis=1, inplace = True)



In [ ]:
print(df.Title.value_counts())



In [ ]:
df.sample(20)


In [ ]:
df.Sex.replace(('male','female'),(0,1), inplace = True)
df.Embarked.replace(('S','C','Q'),(0,1,2), inplace = True)
df.Title.replace(('Mr','Miss','Mrs','Dr','Rev','Officer','Royality','Mlle','Mme','Ms','the Countess','Jonkheer'),(0,1,2,3,4,5,6,7,8,9,10,11), inplace = True)

In [ ]:
df.Title.value_counts()

In [ ]:
corr = df.corr()
corr.Survived.sort_values(ascending=False)

In [ ]:
from sklearn.model_selection import train_test_split

y = df["Survived"]
x = df.drop(['Survived', 'PassengerId'], axis = 1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1)

In [ ]:


randomforest = RandomForestClassifier()
randomforest.fit(x_train, y_train)
y_pred = randomforest.predict(x_test)
acc_randomforest = round(accuracy_score(y_pred, y_test)* 100, 2)
print("Accuracy: {}" .format(acc_randomforest))
pickle.dump(randomforest, open('train.csv', 'wb'))

In [ ]:
df_test =  pd.read_csv('test.csv')
df_test["Title"] = df_test['Name'].map(lambda x : get_title(x))
df_test['Title'] = df_test.apply(shorter, axis=1)
ids = df_test['PassengerId']
df_test['Age'].fillna(df['Age'].median(), inplace=True)
df_test['Fare'].fillna(df['Fare'].median(), inplace=True)
df_test['Embarked'].fillna("S", inplace=True)
df_test.drop("Cabin", axis=1, inplace = True)

df_test.drop("Ticket", axis=1, inplace = True)
df_test.drop("Name", axis=1, inplace = True)
df_test.Sex.replace(('male','female'),(0,1), inplace = True)
df_test.Embarked.replace(('S','C','Q'),(0,1,2), inplace = True)
df_test.Title.replace(('Mr','Miss','Mrs','Dr','Rev','Officer','Royality','Mlle','Mme','Ms','the Countess','Jonkheer'),(0,1,2,3,4,5,6,7,8,9,10,11), inplace = True)



In [ ]:
predictions = randomforest.predict(df_test)
output = pd.DataFrame({'PassengerId' : ids, 'Survived' : predictions})
output.to_csv('submission.csv', index=False)